# DTM Example

This is adapted from https://github.com/piskvorky/gensim/blob/develop/docs/notebooks/dtm_example.ipynb


In [1]:
from gensim import corpora, utils
from gensim.models.wrappers.dtmmodel import DtmModel
import numpy as np

In [2]:
import json

# load the tweets
documents = []
time_seq = []
weeknum = ''
with open('../data/c_twitter.json') as f_in:
    for i, l in enumerate(f_in):
        c_twt = json.loads(l)
        # c_twt: {'weeknum': str, 'c_text': str, 'tags': [str]}
        documents.append(bytes(c_twt['c_text'], 'utf-8').split())
        
        if weeknum != c_twt['weeknum']:
            time_seq.append(i)
            weeknum = c_twt['weeknum']
            
time_seq = time_seq[1:]
time_seq.append(len(documents))

acc = 0
for i in range(1, len(time_seq)):
    acc = acc + time_seq[i-1]
    time_seq[i] = time_seq[i] - acc

print(time_seq)

[2, 103272, 56210, 90675, 209467, 41446, 14002, 43918, 188631, 109273, 54007, 25103, 84203, 52225]


In [ ]:
class DTMcorpus(corpora.textcorpus.TextCorpus):

    def get_texts(self):
        return self.input

    def __len__(self):
        return len(self.input)

corpus = DTMcorpus(documents)

In [ ]:
dtm_binary_path = '/high_spd_work/sm_w2v/tools/dtm/dtm/dtm'
model = DtmModel(dtm_binary_path, corpus, time_seq, num_topics=10,
                 id2word=corpus.dictionary, initialize_lda=True)

In [ ]:
topics = model.show_topic(topicid=1, time=1, topn=10)

In [ ]:
topics

In [ ]:
# currently running out of memory
# do visualization of topics over time